In [1]:
import pandas as pd

# CSV dosyasının yolu (dosya konumunu buraya yaz)
csv_path = "C:\\Users\\90545\\OneDrive\\Masaüstü\\LESSONS\\BITIRMEPROJESI\\ankara_temperature_2025.csv"

# CSV dosyasını oku
df = pd.read_csv(csv_path)

# İlk 5 satırı görüntüle
print(df.head())

             Timestamp  Temperature (°C)
0  2025-01-01 00:00:00         -7.421593
1  2025-01-01 00:15:00         -7.789700
2  2025-01-01 00:30:00         -6.701143
3  2025-01-01 00:45:00         -5.612274
4  2025-01-01 01:00:00         -6.320769


In [3]:
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd

# Basit örnek veri (zaman serisi)
data = [10, 12, 13, 15, 18, 20, 19, 17]

# Model tanımı (p=2, d=1, q=1 gibi)
model = ARIMA(data, order=(2, 1, 1))

# Modeli eğit
model_fit = model.fit()

# Bir adım ileriye tahmin yap
forecast = model_fit.forecast(steps=1)
print("Tahmin:", forecast)


Tahmin: [16.00324689]


C:\Users\90545\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


In [9]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# CSV dosyasını oku (yolu senin bilgisayarına göre güncelle)
df = pd.read_csv("C:\\Users\\90545\\OneDrive\\Masaüstü\\LESSONS\\BITIRMEPROJESI\\ankara_temperature_2025.csv", parse_dates=["Timestamp"])

# Sıcaklık sütununu al
temps = df["Temperature (°C)"].iloc[:5]

# İlk 4 değeri model için kullan
model = ARIMA(temps[:4], order=(2, 1, 0))
model_fit = model.fit()

# Bir sonraki değeri tahmin et
forecast = model_fit.forecast(steps=1)
print("Tahmin edilen sıcaklık (2025-01-01 01:15:00):", forecast.iloc[0])

# Gerçek değerle karşılaştır
print("Gerçek sıcaklık değeri:", temps.iloc[4])


Tahmin edilen sıcaklık (2025-01-01 01:15:00): -5.979713184989502
Gerçek sıcaklık değeri: -6.320769173101823


In [19]:
import pandas as pd
from datetime import timedelta
from sklearn.linear_model import LinearRegression
import numpy as np

# 1) Veriyi oku ve index’i Timestamp yap
df = pd.read_csv(
    "C:\\Users\\90545\\OneDrive\\Masaüstü\\LESSONS\\BITIRMEPROJESI\\ankara_temperature_2025.csv",
    parse_dates=["Timestamp"]
)
df = df.set_index("Timestamp").sort_index()

results = []

# 2) Her bir timestamp için (4 gün gerisinden başlayarak) tahmin ve gerçek değeri hesapla
for ts, row in df.iterrows():
    # 2a) Henüz 4 tam gün geçmiş değilse atla
    if ts - df.index[0] < timedelta(days=4):
        continue

    # 2b) Önceki 4 günün aynı saat-dakika ölçümlerini topla
    X = []
    y = []
    ok = True
    for days_ago in range(1, 5):  # 1,2,3,4 gün önce
        past = (ts - timedelta(days=days_ago)).replace(
            hour=ts.hour, minute=ts.minute,
            second=0, microsecond=0
        )
        try:
            y.append(df.at[past, "Temperature (°C)"])
            X.append([days_ago])
        except KeyError:
            ok = False
            break

    if not ok:
        # veri eksik; bu timestamp için tahmin atlanıyor
        continue

    # 2c) Lineer Regresyon modelini eğit ve tahmin yap
    model = LinearRegression()
    model.fit(np.array(X), np.array(y))
    pred = model.predict(np.array([[0]]))[0]  # 0 gün önce = bugünün değeri

    # 2d) Sonucu kaydet
    results.append({
        "Timestamp": ts,
        "Predicted (°C)": pred,
        "Actual (°C)": row["Temperature (°C)"]
    })

# 3) Sonuçları DataFrame’e çevir ve yazdır
res_df = pd.DataFrame(results).set_index("Timestamp")
print(res_df)

# --- Opsiyonel: sonuçları CSV’ye aktar ---
# res_df.to_csv("prediction_vs_actual.csv")


                     Predicted (°C)  Actual (°C)
Timestamp                                       
2025-01-05 00:00:00       -7.660737    -6.995472
2025-01-05 00:15:00       -8.504368    -9.578730
2025-01-05 00:30:00       -6.776692    -6.374668
2025-01-05 00:45:00       -6.205233    -7.524282
2025-01-05 01:00:00       -8.105248    -8.390936
...                             ...          ...
2025-12-31 22:45:00       -5.329708    -4.422674
2025-12-31 23:00:00       -5.407373    -1.785367
2025-12-31 23:15:00       -2.388450    -3.321656
2025-12-31 23:30:00       -4.275490    -4.093892
2025-12-31 23:45:00       -2.794480    -5.076525

[34656 rows x 2 columns]
